In these notes, we consider a simple mixed integer linear program (MILP) and derive a Benders decomposition scheme to solve it.

In [44]:
using Pkg; Pkg.activate(dirname(@__DIR__))
using JuMP
using HiGHS

  Activating project at `~/Code/power-systems-optimization`


In [45]:
f = [1, 4];
c = [2, 3];
e = [-2; -3];
A = [1 -3; -1 -3];
E = [1 -2; -1 -1];

The full monolithic model is as follows:

In [46]:
monolithic_model = Model(HiGHS.Optimizer);
@variable(monolithic_model, x[1:2] >= 0, Int);
@variable(monolithic_model, y[1:2] >= 0);
@constraint(monolithic_model, A*x + E*y .<= e);
@objective(monolithic_model,Min, f'*x + c'*y);
latex_formulation(monolithic_model)

$$ \begin{aligned}
\min\quad & 2 y_{1} + 3 y_{2} + x_{1} + 4 x_{2}\\
\text{Subject to} \quad & x_{1} - 3 x_{2} + y_{1} - 2 y_{2} \leq -2\\
 & -x_{1} - 3 x_{2} - y_{1} - y_{2} \leq -3\\
 & x_{1} \geq 0\\
 & x_{2} \geq 0\\
 & y_{1} \geq 0\\
 & y_{2} \geq 0\\
 & x_{1} \in \mathbb{Z}\\
 & x_{2} \in \mathbb{Z}\\
\end{aligned} $$

In [47]:
optimize!(monolithic_model)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 3e+00]
  Cost   [1e+00, 4e+00]
  Bound  [0e+00, 0e+00]
  RHS    [2e+00, 3e+00]
Presolving model
2 rows, 4 cols, 8 nonzeros  0s
2 rows, 4 cols, 8 nonzeros  0s

Solving MIP model with:
   2 rows
   4 cols (0 binary, 2 integer, 0 implied int., 2 continuous)
   8 nonzeros
MIP-Timing:     7.8e-05 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0    

In [48]:
value.(monolithic_model[:x])

2-element Vector{Float64}:
 0.0
 1.0

Because we are considering a small example problem, the solver was able to quickly compute a solution. However, real-life mixed integer linear programs involve tens of thousands of integer decisions, pushing even the best commercial solvers to their computational limits. 

Note that if we fix the values of integer variables $x_1$ and $x_2$, we obtain a continuous linear program which can be easily solved. For this reason, Benders decomposition is often implemented to solve MILPs, considering integer decisions as complicating variables. 

Let's initialize the master (or planning) problem:

In [49]:
master = Model(HiGHS.Optimizer);
@variable(master, x[1:2] >= 0, Int);
@variable(master,θ>=-1000) #### Initial lower bound on the operational cost (if operational cost is always positive, this can be set to 0)
@objective(master,Min, f'*x + θ);

latex_formulation(master)

$$ \begin{aligned}
\min\quad & x_{1} + 4 x_{2} + θ\\
\text{Subject to} \quad & x_{1} \geq 0\\
 & x_{2} \geq 0\\
 & θ \geq -1000\\
 & x_{1} \in \mathbb{Z}\\
 & x_{2} \in \mathbb{Z}\\
\end{aligned} $$

Then, the subproblem is defined including auxiliary variables $z$ that act as local copies of the master variables:

In [50]:
subprob = Model(HiGHS.Optimizer);
@variable(subprob, z[1:2] >= 0);
@variable(subprob, y[1:2] >= 0);
@constraint(subprob,A*z + E*y .<= e);
@objective(subprob, Min, c'*y);
latex_formulation(subprob)

$$ \begin{aligned}
\min\quad & 2 y_{1} + 3 y_{2}\\
\text{Subject to} \quad & z_{1} - 3 z_{2} + y_{1} - 2 y_{2} \leq -2\\
 & -z_{1} - 3 z_{2} - y_{1} - y_{2} \leq -3\\
 & z_{1} \geq 0\\
 & z_{2} \geq 0\\
 & y_{1} \geq 0\\
 & y_{2} \geq 0\\
\end{aligned} $$

The Benders decomposition algorithm, starts from solving the master problem:

In [51]:
optimize!(master)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Cost   [1e+00, 4e+00]
  Bound  [1e+03, 1e+03]
Presolving model
0 rows, 0 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -1000           -1000              0.00%        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      -1000
  Dual bound        -1000


Using the solution of the master, we define a lower bound to the optimal value of our original MILP:

In [52]:
LB = objective_value(master)

-1000.0

and also guesses for the master variables:

In [53]:
xk = value.(master[:x])

2-element Vector{Float64}:
 0.0
 0.0

We fix these guesses in the subproblem adding the constraints:

In [54]:
fix.(subprob[:z],xk;force=true)
FixRef.(subprob[:z])

2-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.VariableIndex, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 z[1] = 0
 z[2] = 0

And solve it:

In [55]:
optimize!(subprob)

Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 3e+00]
  Cost   [2e+00, 3e+00]
  Bound  [0e+00, 0e+00]
  RHS    [2e+00, 3e+00]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
2 rows, 2 cols, 4 nonzeros  0s
Presolve : Reductions: rows 2(-0); columns 2(-2); elements 4(-4)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2(5) 0s
          2     7.6666666667e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 2
Objective value     :  7.6666666667e+00
Relative P-D gap    :  1.1584935909e-16
HiGHS run time      :          0.00


We can now compute an upper bound to the optimal value of the original MILP as: 

In [56]:
UB = f'*xk + objective_value(subprob)

7.666666666666666

With lower and upper bounds, we can compute the optimality gap, which gives a conserative estimate of the degree of sub-optimality of our current best guess:

In [57]:
gap = (UB-LB)/abs(UB)

131.43478260869566

To improve our guesses, we need to include additional information into the master problem. 

Hence, we derive the so called optimality cuts, which are based on the dual solutions associated with the constraints fixing the values of the master variables in the subproblems:

In [58]:
λ = dual.(FixRef.(subprob[:z]))

2-element Vector{Float64}:
 -2.0
 -8.0

Having only one subproblem, we add one optimality cut per iteration to the master problem, which is given by:

$$\theta \geq f_k + \lambda^T(x - x_k)$$

where $f_k$ is the objective value of the subproblem obtained fixing variables $z$ to the value $x_k$.

Update the master problem by implementing the optimality cut above:

Then, repeat the above steps until you reach a zero gap. 

At each iteration: 

- Solve the upadted master problem
- Fix the values of the master variables in the subproblem
- Solve the subproblem to obtain an upper bound and compute the gap 
- If it is zero, then you are done! 
- If not zero, compute new optimality gaps and add them to the master problem, then repeat.

Then submit the optimal value you get and the final solution for vector $x$. Report also on how many iterations it took, and write out the cuts computed along the way.